In [ ]:
%load_ext autoreload
%autoreload 2
import networkx as nx
import community.community_louvain as community_louvain
import funkcie as fn
fn.setup_directories()

#### NAČÍTANIE DÁT

In [ ]:
G = fn.load_data("facebook_combined.txt")
pos = nx.spring_layout(G, seed=42)
fn.draw_graph_raw(G, pos)

Načítavam dáta z: facebook_combined.txt
-> Uzlov: 4039
-> Hrán: 88234
Počítam layout (môže to chvíľu trvať)...


#### MODEL LOUVAIN

In [ ]:
# 1. Spustenie algoritmu
partition_louvain = community_louvain.best_partition(G, resolution=1.0)

# 2. Výpočet metrík
mod_louvain = fn.calculate_modularity(G, partition_louvain)
count_louvain = len(set(partition_louvain.values()))

print(f"Louvain -> Modularita: {mod_louvain:.4f}, Počet komunít: {count_louvain}")

# 3. Vizualizácia
fn.draw_communities(G, partition_louvain, pos, "Louvain Algorithm", "img/louvain.png")

Louvain -> Modularita: 0.8349, Komunít: 15


#### MODEL LPA

In [ ]:
# 1. Spustenie algoritmu
communities_lpa = list(nx.algorithms.community.label_propagation_communities(G))

# 2. Konverzia na partition formát
partition_lpa = fn.communities_to_partition(communities_lpa)

# 3. Výpočet metrík
mod_lpa = fn.calculate_modularity(G, partition_lpa)
count_lpa = len(communities_lpa)

print(f"LPA -> Modularita: {mod_lpa:.4f}, Počet komunít: {count_lpa}")

# 4. Vizualizácia
fn.draw_communities(G, partition_lpa, pos, "Label Propagation (LPA)", "img/lpa.png")

LPA -> Modularita: 0.7368, Komunít: 44


#### VYHODNOTENIE

In [5]:
# Zozbieranie výsledkov do zoznamu
results = [
    {"Model": "Louvain", "Modularita": round(mod_louvain, 4), "Počet komunít": count_louvain},
    {"Model": "LPA",     "Modularita": round(mod_lpa, 4),     "Počet komunít": count_lpa}
]

# Uloženie a zobrazenie tabuľky
df_results = fn.save_results_table(results)
df_results

,Model,Modularita,Počet komunít
0,Louvain,0.8349,15
1,LPA,0.7368,44
